# 엘라스틱서치 환경설정

1. vector-scoring을 위해 7.9.0 버젼 설치
- https://www.elastic.co/kr/downloads/past-releases/elasticsearch-7-9-0
2. cmd 에서 실행
- 다운로드된 위치 cd elasticsearch-7.9.0\bin 
- elasticsearch.bat
- 실행확인 후 로컬:9200 에서 작동 확인
3. 형태소분석기 플러그인 설치(다만, 맵핑에는 들어갔으나 사용은 안됨)
- elasticsearch-7.9.0\bin\elasticsearch-plugin install analysis-nori
4. Fast Elasticsearch Vector Scoring 플러그인 설치
- https://soyoung-new-challenge.tistory.com/77 해당 블로그 참조
- git clone -b es-7.9.0 https://github.com/lior-k/fast-elasticsearch-vector-scoring.git
- cd fast-elasticsearch-vector-scoring
- mvn package
- elasticsearch-7.9.0\bin\elasticsearch-plugin install file:///fast-elasticsearch-vector-scoring/target/releases/elasticsearch-binary-vector-scoring-7.9.0.zip (파일위치 조정필요)

# 엘라스틱서치

In [ ]:
!pip install elasticsearch==7.9.0

In [ ]:
from elasticsearch import Elasticsearch
import pandas as pd
es = Elasticsearch(["http://127.0.0.1:9200"])

In [ ]:
# 인덱스 삭제
# es.indices.delete(index='review_list', ignore=[400, 404])

In [ ]:
# 인덱스 생성 및 설정
es.indices.create(
    index='review_list',
    body={
        "settings": {
            "index": {
                "analysis": {
                    "analyzer": {
                        "my_analyzer": {
                            "type": "custom",
                            "tokenizer": "nori_tokenizer" #한글 토크나이저
                        }
                    }
                }
            }
        },
        "mappings": {
                "properties": {
                    "ht_id": {
                        "type": "keyword"
                    },
                    "ht_name": {
                        "type": "keyword"
                    },
                    "cat_id": {
                        "type": "keyword"
                    },
                    "cat_name": {
                        "type": "keyword"
                    },
                    "date": {
                        "type": "date",
                        "format": "yyyy.mm.dd"
                    },
                    "review": {
                        "type": "text",
                        "analyzer": "my_analyzer"
                    },
                    "rev_vec": {
                      "type": "dense_vector",
                      "dims": 768
                    },
                    "label": {
                        "type": "integer"
                    },
                     "grade": {
                        "type": "keyword"
                    },
                    "gu": {
                        "type": "keyword"
                    },
                    "station": {
                        "type": "keyword"
                    },
                   "station2": {
                        "type": "keyword"
                    },
                    "word1": {
                        "type": "keyword"
                    },
                    "word2": {
                        "type": "keyword"
                    },
                    "word3": {
                        "type": "keyword"
                    }
                }
            }
        }
    
)

## 데이터 입력
- 백터화된 리스트가 파일로 저장되면서 문자열로 읽힘
- 다시 백터화된 리스트로 변환하는 과정 필요

In [ ]:
data=pd.read_csv('./엘라스틱DB_최종.csv')

In [ ]:
from tqdm import tqdm
import ast #문자열리스트를 파이썬리스트로 변환시키는 라이브러리
for i in tqdm(range(data.shape[0])):
    k=ast.literal_eval(data['embedding'][i])
    data['embedding'][i] = k

In [ ]:
def insertData(df):
    es = Elasticsearch(["http://127.0.0.1:9200"])
    for i in tqdm(range(df.shape[0])):
        doc = {
            "ht_id" : df['ht_id'][i],
            "ht_name" : df['ht_name'][i],
            "cat_id" : df['cat_id'][i],
            "date" : df['date'][i],
            "review" : df['review'][i],
            "rev_vec" : df['embedding'][i],
            "label" : df['label'][i],
            "grade" : df['등급'][i],
            "gu" : df['구'][i],
            "station" : df['지하철역'][i],
            "station2" : df['지하철역2'][i],
            "word1" : df['지명'][i],
            "word2" : df['지명2'][i],
            "word3" : df['지명3'][i]
        }
        es.index(index="review_list",  body=doc)
insertData(data)

## 검색

In [ ]:
import re
from konlpy.tag import Okt
from sentence_transformers import SentenceTransformer, util
from tqdm import tqdm
import json
model = SentenceTransformer('snunlp/KR-SBERT-V40K-klueNLI-augSTS') 
with open('./Ko_stopwords.txt', 'r', encoding='utf-8') as f:
    stopwords = f.readlines()
    stopwords = [x.strip() for x in stopwords]
    
def preprocess_text(text):
    mecab = Okt()
    text = re.sub('[^0-9가-힣ㄱ-ㅎㅏ-ㅣ\s]', '', text)
    tokens = mecab.morphs(text)
    tokens = [word for word in tokens if word not in stopwords]
    preprocessed_text = ' '.join(tokens)
    return preprocessed_text

ht_name_pattern = r'\b(24게스트하우스서울시청점|57명동호스텔|AC호텔바이메리어트서울강남|BB홍대호스텔|D.H네상스호텔|DW디자인레지던스|ENA스위트호텔남대문|G2호텔명동|G3호텔충무로|Glue호텔|G게스트하우스이태원인서울|HAvenue이대점|Ibc호텔|JK블라썸|JW메리어트동대문스퀘어|JW메리어트호텔서울|K-그랜드호스텔동대문|K게스트하우스동대문프리미엄|L7강남바이롯데|L7명동바이롯데|L7홍대바이롯데|N285호텔인사동|SR호텔사당점|SR호텔서울마곡|Stay7명동점|WD호텔|Zip호텔|가산더스테이호텔|가산마인드호텔|강남렉시|강남멜리샤호텔|강남아르누보씨티|강남제리스플래닛|강남캠퍼스|건대드가자|건대컬리넌-1호점|건대컬리넌-2호점|건대호텔KWorld|골드리버호텔|골든서울호텔|그랜드머큐어앰배서더호텔앤레지던스서울용산|그랜드워커힐서울|그랜드인터컨티넨탈서울파르나스|그랜드하얏트서울|그리드인호텔|글래드강남코엑스센터|글래드마포|글래드여의도|길동IMT1,2|나인트리프리미어호텔명동2|나인트리프리미어호텔인사동|나인트리호텔동대문|나인트리호텔명동|남산힐호텔|노보텔스위트앰배서더서울용산|노보텔앰배서더서울강남|노보텔앰배서더서울동대문호텔앤레지던스|노보텔앰배서더서울용산|노원필름37.2호텔|뉴국제호텔|뉴서울호텔|대영호텔명동|더가든호텔|더리버사이드호텔|더리센츠동대문호텔|더블라썸연남게스트하우스|더스테이트선유|더케이호텔서울|더클래식500이그제큐티브레지던스펜타즈|더퍼스트스테이호텔|더플라자서울오토그래프컬렉션|도미인서울강남|독산3SHOTEL|독산호텔인카페|드림게스트하우스|디아티스트호텔역삼점|디어스명동|딜라이트호텔잠실|라까사호텔서울|라마다서울동대문|라마다서울신도림|라마다호텔앤스위트서울남대문|라비타호텔|라이즈오토그래프컬렉션바이메리어트|라인호텔명동|레스케이프호텔|레이크호텔|로사나호텔|로얄스퀘어호텔|로이넷호텔서울마포|롯데시티호텔구로|롯데시티호텔김포공항|롯데시티호텔마포|롯데시티호텔명동|롯데호텔서울|롯데호텔서울이그제큐티브타워|롯데호텔월드|마마스앤파파스홍대게스트하우스|머큐어앰배서더서울홍대|메리어트이그제큐티브아파트먼트서울|메이플러스서울동대문|메이필드호텔서울|명동뉴스테이인|명동멀린호텔|명동메트로호텔|명동스타힐스호텔|명동아트래블홈|목시서울인사동|몬드리안서울이태원|밀리오레호텔서울명동|반얀트리클럽앤스파서울|베니키아노블호텔|베스트웨스턴프리미어강남|베이튼호텔|보코서울강남|불광포레스타|비스타워커힐서울|사당MRG|사당티트리호텔|사당호텔카일|사보이호텔|서머셋팰리스|서울가든호텔|서울대54번가|서울맘게스트하우스|서울신라호텔|서울신촌위고인호스텔|서울앤호텔동대문|서울올림픽파크텔|서울웨스틴조선호텔|서울킴스테이9|서초라바|선릉그레이호텔|센터마크호텔서울|성신여대역더월|성신여대역샤미소|성신여대호텔디아티스트|세종호텔|센트럴관광호텔|소설호텔|소테츠프레사인서울명동|소테츠호텔즈더스프라지르서울동대문|소테츠호텔즈더스프라지르서울명동|소피텔앰배서더서울|솔라리아니시테츠명동|수송게스트하우스|수유호텔클래시|스위스그랜드호텔|스타즈호텔독산|스타즈호텔명동2호점|스탠포드호텔명동|스탠포드호텔서울|스탭인명동2|스테이비명동호텔|스테이호텔강남|슬로우스테이DA|시그니엘서울|신라스테이광화문|신라스테이구로|신라스테이마포|신라스테이삼성|신라스테이서대문|신라스테이서초|신라스테이역삼|신림돈키호텔|신림르네상스|신천A+무인호텔|신천포레스타1|신천포레스타2|신천호텔더캐슬-잠실새내점|신천엔유|신촌MAC|신촌가을|신촌넘버25|신촌라뉘|신촌라싸|신촌루씨르|신촌림|신촌모모-구이젠|신촌신디호텔|신촌앨리|신촌어반노드|신촌포레스타|써미트호텔|아리랑힐호텔동대문|아마레호텔종로|아만티호텔서울|안다즈서울강남|안테룸서울|알로프트서울강남|알로프트서울명동|앰배서더서울풀만호텔|야코리아호스텔강남점|야코리아호스텔동대문|약수프린스모텔|어반플레이스강남|업플로호스텔|에버8서비스레지던스|에이든바이베스트웨스턴청담|에이큐브호텔|여의도메리어트호텔|여의도코보스호텔|역삼PREMIERXYM|역삼리치웰|역삼린|역삼벤|역삼브라운도트|역삼스타호텔|역삼인트로호텔|역삼컬리넌|역삼호텔스타프리미어|영등포Blvd호텔오라|영등포GMS호텔|영등포그곳에|영등포데쟈트|영등포부띠크HotelSB|영등포호텔갤럭시|영등포호텔더휴|영등포호텔브릿지|오라카이대학로호텔|오라카이인사동스위츠|오라카이청계산호텔|오리엔스호텔앤레지던스|오월호텔|오크우드프리미어코엑스센터|오클라우드호텔|왕십리아모렉스|왕십리컬리넌|왕십리포레스타|용산엘르인|웨스턴코업동대문레지던스|은평씨에스에비뉴호텔|을지로코업레지던스|이비스스타일앰배서더강남|이비스스타일앰배서더서울명동|이비스스타일앰배서더서울용산|이비스앰배서더명동|이비스앰배서더인사동|이코노미호텔명동프리미어점|이태원옐로우게스트하우스|이태원인|인사동미니호텔|인사동호스텔|인사동호텔썬비|인터시티서울|인터컨티넨탈서울코엑스|임피리얼팰리스부티크호텔|잠실(방이)호텔더캐슬1호점|잠실2.4호텔|잠실HLHotel|잠실Stayhotel|잠실라비호텔|잠실루이체|잠실셀레네|잠실와우|잠실첼로|잠실호텔넘버25|잠실호텔톰지|잠실포레스타호텔|장안시그니처|조선팰리스서울강남럭셔리컬렉션호텔|종로(5가)호텔KWORLD|종로HOTELLABOUM|종로M&LUCKYHOTEL|종로THEMAYHOTEL|종로더포스트호텔|종로부티크호텔K|종로시네마|종로아비숑|종로헤르츠|종로호스텔토미|종로호텔라와|종로호텔팝리즈프리미어|창동론스타|천호HOTELH|카파스호텔|카푸치노호텔|칼리스타호텔|케니스토리인종로|케이스타메트로호텔|케이팝호텔서울역점|켄싱턴호텔여의도|코리아나호텔|코코모호텔|코코아게스트하우스|코트야드메리어트서울남대문|코트야드메리어트서울보타닉파크|코트야드메리어트서울타임스퀘어|콘래드서울|크라운파크호텔서울|크레토호텔명동|토요코인서울강남|토요코인서울동대문2|토요코인서울영등포|통통쁘띠호텔|트레블로지동대문|트레블로지명동시티홀호텔|트레블로지명동을지로호텔|트리아호텔|파라스파라서울|파로스호텔|파크하비오호텔|파크하얏트서울|파티오세븐호텔|퍼시픽호텔|페어몬트앰배서더서울|페어필드바이메리어트서울|포시즌스호텔서울|포포인츠바이쉐라톤서울강남|포포인츠바이쉐라톤서울구로|포포인츠바이쉐라톤조선서울명동|포포인츠바이쉐라톤조선서울역|프레이저플레이스센트럴서울|프레지던트호텔|하모니호텔|하우재이태원|하이서울유스호스텔|해밀톤호텔|핸드픽트호텔|헨나호텔서울명동|호스텔코리아|호텔28명동|호텔8아워즈|호텔DM|호텔U5|호텔가온골든파크동대문|호텔국도명동|호텔그레이스리서울|호텔나포레|호텔뉴브|호텔더디자이너스DDP|호텔더디자이너스건대프리미어|호텔더디자이너스동대문|호텔더디자이너스리즈강남프리미어|호텔더디자이너스서울역|호텔더디자이너스종로|호텔더디자이너스청량리|호텔더디자이너스홍대|호텔로프트|호텔리베라서울|호텔리베라청담|호텔리안|호텔릿서울역|호텔마누서울|호텔미도명동|호텔미드시티명동|호텔베뉴지|호텔베르누이서울|호텔부티크9|호텔삼정|호텔선샤인서울|호텔스카이파크동대문1호점|호텔스카이파크명동1호점|호텔스카이파크명동2호점|호텔스카이파크명동3호점|호텔스카이파크센트럴명동점|호텔스카이파크킹스타운동대문점|호텔스테이인|호텔아이린시티|호텔아트리움종로|호텔엔트라|호텔엠펠리체|호텔유리앤|호텔인나인강남|호텔인피니|호텔쿠레타케소인사동|호텔크레센도서울|호텔토마스명동|호텔페이토강남|호텔페이토삼성|호텔포코|호텔프린스서울|호텔피제이명동|홀리데이인익스프레스서울홍대|홍대나비호스텔|홍대나인브릭호텔|홍대더휴식아늑|홍대써니힐게스트하우스|화곡HOTELN|화곡Mshotel|화곡VOLL|화곡블루힐|화곡소설스미스|화곡해담채|화웬하우스hq|힐튼가든인서울강남)\b'
gra_pattern = r'\b(게스트하우스|모텔|5성|호스텔|4성|2성|3성)\b'
gu_pattern = r'\b(은평구|마포구|노원구|강남구|광진구|강북구|강서구|송파구|구로구|도봉구|금천구|서초구|종로구|동대문구|중구|영등포구|동작구|관악구|용산구|서대문구|성북구|성동구|강동구)\b'
sta_pattern = r'\b(합정역|선릉역|삼성역|길동역|역삼역|발산역|숙대입구역|구로디지털단지역|여의나루역|한강진역|장한평역|연주역|사당역|신논현역|남부터미널역|신천역|여의도역|을지로4가역|성신여대역|역삼역역|고속터미널역|성신여대입구역|신길역|신림역|광화문역|교대역|동대문역사문화공원역|성수역|공덕역|충무로역|녹사평역|오류동역|서울역|디지털미디어시티역|잠실역|동대입구역역|이대역|을지로입구|가산디지털단지역|이태원역|군자역|언주역|김포공항역|수유역|광화문|약수역|마포역|상왕십리역|왕십리역|독산역|불광역|잠실새내역|길동사거리.강동세무서역|송정역|몽촌토성역|회현역|남영역|몽촌토성역역|선정릉역|창동역|청담역|강남역|북한산우이역|건대입구역|삼성중앙역|동대입구역|천호역|시청역|버티고개역|광나루역|국회의사당역|종합운동장역|종각역|종로5가역|동묘앞역|학동역|을지로입구역|삼각지역|상수역|영등포역|서대문역|석촌고분역|오목교역|노원역|을지로|3가역|안국역|염창역|압구정역|장지역|당산역|홍대입구역|종로3가역|마곡나루역|청구역|봉은사역|구로역|청계산입구역|양재역|을지로3가역|신사역|명동역|홍제역|서울대입구역|신촌역|선유도역|용산역|신대방삼거리역|청량리역)\b'

wo1_pattern = r'\b(청담|선릉|신길|회현|서울|종합운동장|사당|독산|길동사거리|동대입구|오목교|신천|길동|삼성|삼전|신설동|을지로4가|국회의사당|강남|이태원|녹사평|염창|서대문|성신여대|수유|영등포|공덕|광나루|홍제|구로|서울대입구|안국|왕십리|충무로|동묘앞|발산|약수|오류동|이대|장지|건대|천호|김포공항|상수|구로디지털단지|봉은사|삼각지|몽촌토성|노원|장한평|신촌|까치산|석촌고분|을지로|3가|고터|숙대입구|학동|청계산입구|마포|가산디지털단지|남부터미널|청량리|남영|녹번|신대방삼거리|홍대|광화문|디지털미디어시티|광화|연주|양재|북한산우이|을지로입구|한강진|을지로3가|언주|합정|잠실새내|여의도|종각|송정|삼성중앙|당산|교대|여의나루|신논현|용산|잠실|을지로입|명동|신림|압구정|성수|종로5가|시청|청구|역삼|신사|불광|선정릉|군자|선유도|종로3가|상왕십리|창동|동역사|버티고개|마곡나루)\b'
wo2_pattern = r'\b(동대문역사문화공원|몽촌토성|명동|충무로|양재|언주|신도림|신길|회현|을지로입구|봉은사|공덕|잠실|잠실새내|선릉|시청|신사|종로3가|을지로입구|광화문)\b'
wo3_pattern = r'\b(송파|광진|도봉|서초|금천|종로|동대문|노원|강서|은평|동작|관악|강남|용산|성동|서대문|영등포|성북|강동|강북|구로|마포)\b'


In [ ]:
search = '아이랑 놀기 좋은곳'
query_texts = preprocess_text(search)

query_texts = re.sub(r'(\d+) 성', r'\1성', query_texts)  # 3 성 같은 숫자+성 붙이기 
query_text = query_texts

#호텔 이름추출 시에는 바로 페이지 리턴
matches_name = re.findall(ht_name_pattern, search) 

matches1 = re.findall(gra_pattern, query_text)
matches2 = re.findall(gu_pattern, query_text)
matches3 = re.findall(sta_pattern, query_text)
matches4 = re.findall(wo1_pattern, query_text)
matches5 = re.findall(wo2_pattern, query_text)
matches6 = re.findall(wo3_pattern, query_text)

# 만약 해당되는게 있다면 필터링하고 문장에서 제외
if matches1:
    grade = matches1[0]
    query_text = re.sub(grade,'',query_text)
    query_text = preprocess_text(query_text)
else : grade = None
if matches2:
    gu = matches2[0]
    query_text = re.sub(gu,'',query_text)
    query_text = preprocess_text(query_text)
else : gu = None
if matches3:
    station = matches3[0]
    query_text = re.sub(station,'',query_text)
    query_text = preprocess_text(query_text)
else : station = None
if matches4:
    word1 = matches4[0]
    query_text = re.sub(word1,'',query_text)
    query_text = preprocess_text(query_text)
else : word1 = None
if matches5:
    word2 = matches5[0]
    query_text = re.sub(word2,'',query_text)
    query_text = preprocess_text(query_text)
else : word2 = None
if matches6:
    word3 = matches6[0]
    query_text = re.sub(word3,'',query_text)
    query_text = preprocess_text(query_text)
else : word3 = None
    
#검색내역에서 전부 조건 필터링에 걸릴 경우 그냥 검색
if len(query_text)<=1: query_text = query_texts 

#코사인유사도를 이용한 검색쿼리
body ="""
    {"query": {"function_score": {"query": {"bool": {"must": [{"match": {"review": "%s"}}], "should": []}},
                                "script_score": {"script": {"source": "cosineSimilarity(params.query_vector, 'rev_vec') + 1.0",
                                                            "params": {"query_vector": %s}}}}}, "size": 50}
""" % (query_text,list(model.encode(query_text)))

if gu:
    body_json = json.loads(body)
    body_json["query"]['function_score']['query']["bool"]["should"].append({"match": {"gu": {"query": gu,"boost": 10.0}}})
    body = json.dumps(body_json)
if station:
    body_json = json.loads(body)
    body_json["query"]['function_score']['query']["bool"]["should"].append({"match": {"station": { "query": station,"boost": 10.0} }})
    body = json.dumps(body_json)
if station:
    body_json = json.loads(body)
    body_json["query"]['function_score']['query']["bool"]["should"].append({"match": {"station2": {"query": station,"boost": 10.0}}})
    body = json.dumps(body_json)
if grade:
    body_json = json.loads(body)
    body_json["query"]['function_score']['query']["bool"]["should"].append({"match": {"grade": {"query": grade,"boost": 10.0}}})
    body = json.dumps(body_json)
if word1:
    body_json = json.loads(body)
    body_json["query"]['function_score']['query']["bool"]["should"].append({"match": {"word1": {"query": word1,"boost": 10.0}}})
    body = json.dumps(body_json)
if word2:
    body_json = json.loads(body)
    body_json["query"]['function_score']['query']["bool"]["should"].append({"match": {"word2": {"query": word2,"boost": 10.0}}})
    body = json.dumps(body_json)
if word3:
    body_json = json.loads(body)
    body_json["query"]['function_score']['query']["bool"]["should"].append({"match": {"word3": {"query": word3,"boost": 10.0}}})
    body = json.dumps(body_json)


res = es.search(index='review_list', body=body) #검색

#---------------------------------------------
# 출력방식
next_ls = []
for i in range(len(res['hits']['hits'])):
    next_ls.append([res['hits']['hits'][i]['_source']['ht_id'],
                    res['hits']['hits'][i]['_score'],
                    res['hits']['hits'][i]['_source']['cat_id'],
                    res['hits']['hits'][i]['_source']['date'],
                    res['hits']['hits'][i]['_source']['review'],
                    res['hits']['hits'][i]['_source']['label'],
                    ]) 

next_df = pd.DataFrame(next_ls,columns=['ht_id','score','cat_id','date','review','label'])
# 호텔아이디 중복은 첫번째꺼 외에 모두 제거
next_df=next_df.drop_duplicates(['ht_id'], keep='first')
# 유사도점수가 15점 이상인 것만 추출
next_df = next_df[next_df['score']>=15]
next_df.reset_index(drop=True, inplace=True)

next_df